# Using Connectivity map data and PharmacoGx R package to find drugs against coronaviruses

Connectivity map or Cmap is resource that uses gene expression data to connect disease transcriptomic signature with those of different types of cells after drug treatment. If the transcriptomic signature of a drug on cell line is opposite from disease signature, it can be expected that this drug could have a positive effect on amelioration of disease.


References:
Subramanian A, et al. A Next Generation Connectivity Map: L1000 Platform And The First 1,000,000 Profiles. Cell. 2017/12/1. 171(6):1437–1452.

Lamb J, et al. The Connectivity Map: using gene-expression signatures to connect small molecules, genes, and disease. Science. 2006/9/29. 313(5795):1929-35, (2006)

Harmonizome is a collection of processed datasets from over 70 major online resources. I downloaded from Harmonizome "GEO Signatures of Differentially Expressed Genes for Viral Infections". For this analysis I used transcriptomics data of lungs from mice that were infected with icSARS CoV (GEO accesion number GSE50000;7th day of infection). For this analysis I havent used transcriptomic data of novel human coronavirus 2019 or SARS-2, but according to the literature all of these viruses (MERS,SARS1 and SARS2) are betacoronaviruses that have origin in bats and SARS-2 is shown to be very similar to SARS-01 according to phylogenic analysis. 

References:
Rouillard AD, Gundersen GW, Fernandez NF, Wang Z, Monteiro CD, McDermott MG, Ma'ayan A. The harmonizome: a collection of processed datasets gathered to serve and mine knowledge about genes and proteins. Database (Oxford). 2016 Jul 3;2016. pii: baw100.
	
Josset L, Menachery VD, Gralinski LE, Agnihothram S et al. Cell host response to infection with novel human coronavirus EMC predicts potential antivirals and important differences with SARS coronavirus. mBio 2013 Apr 30;4(3):e00165-13. PMID: 23631916

Menachery VD, Eisfeld AJ, Schäfer A, Josset L et al. Pathogenic influenza viruses and coronaviruses utilize similar and contrasting approaches to control interferon-stimulated gene responses. mBio 2014 May 20;5(3):e01174-14. PMID: 24846384

Yushun Wan, Jian Shang, Rachel Graham, Ralph S. Baric, Fang Li. Receptor recognition by novel coronavirus from Wuhan: An analysis based on decade-long structural studies of SARS. Journal of Virology, 2020; DOI: 10.1128/JVI.00127-20

Yadi Zhou, Yuan Hou, Jiayu Shen, Yin Huang, William Martin, Feixiong Cheng.Network-based Drug Repurposing for Human Coronavirus
doi: https://doi.org/10.1101/2020.02.03.20020263

For the analysis of Connectivity map data I used PharmacoGx R package.

P Smirnov, Z Safikhani, N El-Hachem , D Wang, A She , C Olsen, MFreeman, H Selby, D Gendoo , P Grossman , A Beck , H Aerts ,
M Lupien, A Goldenberg ,B Haibe-Kains PharmacoGx: an R package for analysis of large pharmacogenomic datasets Bioinformatics (2016).

In [76]:
library(PharmacoGx)

In [77]:
availablePSets()


,PSet.Name,Dataset.Type,Available.Molecular.Profiles,Date.Updated,URL
,<fct>,<fct>,<fct>,<fct>,<fct>
CCLE_2013,CCLE_2013,sensitivity,rna/mutation,Tue Sep 15 18:50:07 2015,https://www.pmgenomics.ca/bhklab/sites/default/files/downloads/CCLE_Nature2013.RData
CCLE,CCLE,sensitivity,rna/rnaseq/mutation/cnv,Thu Dec 10 18:17:14 2015,https://www.pmgenomics.ca/bhklab/sites/default/files/downloads/CCLE.RData
GDSC_2013,GDSC_2013,sensitivity,rna/mutation,Mon Oct 5 16:07:54 2015,https://www.pmgenomics.ca/bhklab/sites/default/files/downloads/CGP_Nature2013.RData
GDSC,GDSC,sensitivity,rna/rna2/mutation/fusion/cnv,Wed Dec 30 10:44:21 2015,https://www.pmgenomics.ca/bhklab/sites/default/files/downloads/GDSC.RData
GDSC1000,GDSC1000,sensitivity,rna,Thu Aug 25 11:13:00 2016,https://www.pmgenomics.ca/bhklab/sites/default/files/downloads/GDSC1000.RData
gCSI,gCSI,sensitivity,rnaseq/cnv,Mon Jun 13 18:50:12 2016,https://www.pmgenomics.ca/bhklab/sites/default/files/downloads/gCSI.RData
FIMM,FIMM,sensitivity,,Mon Oct 3 17:14:00 2016,https://www.pmgenomics.ca/bhklab/sites/default/files/downloads/FIMM.RData
CTRPv2,CTRPv2,sensitivity,,Thu Aug 25 11:15:00 2016,https://www.pmgenomics.ca/bhklab/sites/default/files/downloads/CTRPv2.RData
CMAP,CMAP,perturbation,rna,Mon Sep 21 02:38:45 2015,https://www.pmgenomics.ca/bhklab/sites/default/files/downloads/CMAP.RData


In [78]:
CMAP.sigs <- downloadPertSig("CMAP")

In [80]:
 drug.perturbation <- CMAP.sigs

In [82]:
sars1_up_and_down_genes <-read.csv("icSARS_Cov_Day7_None_GSE50000.csv",stringsAsFactors = F, header = T, sep=";")
rownames(sars1_up_and_down_genes) <- sars1_up_and_down_genes$Feature
sars1<- sars1_up_and_down_genes$Direction
names(sars1) <- rownames(sars1_up_and_down_genes)


In [83]:
res_sars1 <- apply(drug.perturbation[,,c("tstat", "fdr")], 
             2,function(x, sars1){
         return(connectivityScore(x=x, 
                                  y=sars1,
                               method="fgsea", nperm=100))
       },sars1 = sars1)

In [84]:
rownames(res_sars1) <- c("Connectivity", "P_value")
res_sars1 <- t(res_sars1)


[1] "Connectivity" "P_value"

In [95]:
con_score_sars <-res_sars1[order((res_sars1[,1]),decreasing = F),]
p_value_sars <- res_sars1[order((res_sars1[,2]),decreasing = F),]

In [96]:
p_value_sars


,Connectivity,P_value
valproic acid,-0.376625,0.003243431
rifabutin,-0.340380,0.003249652
levomepromazine,-0.329905,0.003363063
promazine,-0.338460,0.003408985
Prestwick-559,-0.332285,0.003453743
maprotiline,-0.345995,0.003483930
nicergoline,-0.297975,0.003540995
GW-8510,-0.365775,0.003643482
lasalocid,-0.287965,0.003653011
pyrvinium,-0.312920,0.003692544


In [93]:
write.csv(p_value_sars,"sars_results.csv", row.names = T)

Based on p value and connectivity score (negative score means that drug reverses the disease signature) I have decided to do 
docking on valproic acid. This drug is FDA-approved for treatment of epilepsy, bipolar disorder  and migraine headeche. 
Valproic acid is also tested in the treatment of AIDS and cancer.
I also in my results get that sirolimus can be used against coronaviruses, and study by Zhou et al get the similar results.
 
References:
Yadi Zhou, Yuan Hou, Jiayu Shen, Yin Huang, William Martin, Feixiong Cheng.Network-based Drug Repurposing for Human Coronavirus
doi: https://doi.org/10.1101/2020.02.03.20020263

O. Trott, A. J. Olson,AutoDock Vina: improving the speed and accuracy of dockingwith a new scoring function,
efficient optimization and multithreading, Journal of Computational Chemistry 31 (2010); 455-461; DOI 10.1002/jcc.21334  

Study of Valproic Acid to Treat HIV Infected Adults https://clinicaltrials.gov/ct2/show/NCT00312546

Vázquez-Calvo A, Saiz JC, Sobrino F, Martín-Acebes MA. Inhibition of enveloped virus infection of cultured cells by valproic acid.
J Virol. 2011;85(3):1267–1274. doi:10.1128/JVI.01717-10